<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# NLP Using the Twitter API: Guided Lab

_Authors: Dave Yerrington (SF)_

---


<img src="https://snag.gy/RNAEgP.jpg" width="600">

### Can we correctly identify which of these two old men tweeted what?

> *Note: this lab is intended to be a guided lab until the independent practice questions.*


## Goals
---

We are going to attempt to classify whether a tweet comes from Trump or Sanders.  This lab involves multiple steps:
- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration
---

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

---

![](https://snag.gy/HPBQbJ.jpg)

Go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/).

> **Note**: For the required website field you can put a placeholder.

After you set up our app, you will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

---

Someone was nice enough to build a Python libary for us. It makes pulling tweets simple: we only need to plug in our keys and start collecting data. The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [1]:
!pip install twitter python-twitter

    100% |████████████████████████████████| 61kB 1.7MB/s 
    100% |████████████████████████████████| 81kB 4.0MB/s 
    100% |████████████████████████████████| 133kB 2.0MB/s 
  Running setup.py bdist_wheel for oauthlib ... - \ done
  Stored in directory: /Users/kiefer/Library/Caches/pip/wheels/84/98/7a/fba7268f61097bea6081cbe5480bc439b38975748ea7684fd5
Successfully built oauthlib


## Some Boring Twitter Rules
---

**Twitter notifies you they will rate limit your requests:**

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rules":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review
---

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys
---

Take note of your application keys you will use to connect to Twitter and mine tweets from the official Bernie Sanders and Donald Trump twitter accounts:

![](https://snag.gy/H1djQK.jpg)

## `TweetMiner` class structure

---

The following code will get you up and running, providing connectivity to twitter. The class has the ability to make requests and can eventually transform the JSON responses into DataFrames.

This is a great example of using object-oriented Python to organize our code!

> **Note:** "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding the rate limit blocks.

### Twitter API key setup

Fill the information below in with the keys for your account.

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the button to generate tokens to get this
- **access_token_secret** - Also available after you generate tokens


In [1]:
import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        'lA6UQplM5sxIAIr83ueUl9sgE',
    'consumer_secret':     'f94t4BD6Vj7aCVkX1qAIVwsP4x69J2vXvm61lTIuwb9GfmdsuP',
    'access_token_key':    '1203210464-dYE1FvoUx1GjVcoyok3U1brWDpELBJEcSNGC1OC',
    'access_token_secret': 'kuVli1j010RPdYTboz9iIyp8QxX6PqRApdi49baLqBgzo'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


ModuleNotFoundError: No module named 'twitter'

In [2]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        'KmN03M1X1pImZ43sqdIu4yfnE',
        'consumer_secret':     'ePlIrIX5VXbZnO7DBu1RbFlw5lOai9dQr9n5TZb6vxnIdrr5Fz',
        'access_token_key':    '185036086-Q7K5IjuSoQZJwSIqD0wyHf6t62iPKatmfaPkriAM',
        'access_token_secret': 'cYpQz3xWHQbLplOj8iSeiNSOmMcsTOXmWcKMrJ9buLj5d'
    }
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.api = api
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False, max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

## Instantiate the class
---

Make sure you pass the keys dictionary and the api as arguments.

**Check:** call the object's `mine_user_tweets()` method, providing a user to pull the tweets of.

In [3]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

In [4]:
sanders = miner.mine_user_tweets(user="berniesanders", max_pages=5)
donald = miner.mine_user_tweets(user="realDonaldTrump", max_pages=5)

In [5]:
print(sanders[0])

{'mined_at': datetime.datetime(2017, 12, 4, 0, 55, 59, 194862), 'created_at': 'Sat Dec 02 18:08:24 +0000 2017', 'retweet_count': 557, 'handle': 'Bernie Sanders', 'tweet_id': 937020642410582017, 'text': 'RT @ninaturner: I’m here in Dayton, Ohio w/ @BernieSanders bringing the message to the people! We are on the move. #GOPTaxBill @OurRevoluti…'}


In [6]:
print(donald[0])

{'mined_at': datetime.datetime(2017, 12, 4, 0, 56, 0, 846116), 'created_at': 'Sun Dec 03 13:36:45 +0000 2017', 'retweet_count': 30904, 'handle': 'Donald J. Trump', 'tweet_id': 937314665176207360, 'text': 'Report: “ANTI-TRUMP FBI AGENT LED CLINTON EMAIL PROBE”  Now it all starts to make sense!'}


### Convert the tweet ouputs to a pandas DataFrame

> *Hint: this is as easy as passing it to the DataFrame constructor!*

In [7]:
pd.DataFrame(sanders).head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Sat Dec 02 18:08:24 +0000 2017,Bernie Sanders,2017-12-04 00:55:59.194862,557,"RT @ninaturner: I’m here in Dayton, Ohio w/ @B...",937020642410582017
1,Wed Nov 29 03:08:23 +0000 2017,Bernie Sanders,2017-12-04 00:55:59.194873,7079,The Republican tax bill is a moral abomination...,935706980643147777
2,Tue Nov 21 01:23:49 +0000 2017,Bernie Sanders,2017-12-04 00:55:59.515570,6022,Attacks against trans people are part of a big...,932781561845608448
3,Fri Nov 17 16:21:31 +0000 2017,Bernie Sanders,2017-12-04 00:55:59.515581,942,RT @viceimpact: .@BernieSanders on why it's ti...,931557926615027713
4,Thu Nov 16 00:13:03 +0000 2017,Bernie Sanders,2017-12-04 00:55:59.828491,918,RT @politico: Tim Kaine calls for eliminating ...,930951816124289024


##  Create the training data

---

Let's get our "mined" data from the Twitter API.  

1. Mine Trump tweets
- Create a tweet DataFrame
- Mine Sanders tweets
- Append the results to our DataFrame

In [8]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = TweetMiner(twitter_keys, api, result_limit=400)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [9]:
trump_df = pd.DataFrame(trump_tweets)
print(trump_df.shape)

(995, 6)


In [10]:
bernie_tweets = miner.mine_user_tweets('berniesanders')

In [11]:
bernie_df = pd.DataFrame(bernie_tweets) 
print(bernie_df.shape)

(1000, 6)


In [12]:
tweets = pd.concat([trump_df, bernie_df], axis=0)
tweets.shape

(1995, 6)

## Any interesting ngrams going on with Trump?
---

Set up a vectorizer from sklearn and fit the text of Trump's tweets with an ngram range from 2 to 4. Figure out what the most common ngrams are.

> **Note:** It's up to you whether you want to remove stopwords or not. How does keeping or removing stopwords affect the results?

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('https co', 445),
 ('of the', 83),
 ('will be', 66),
 ('in the', 63),
 ('to the', 57),
 ('thank you', 56),
 ('for the', 49),
 ('fake news', 42),
 ('tax cuts', 41),
 ('our country', 37),
 ('at the', 34),
 ('to be', 34),
 ('we are', 30),
 ('our great', 29),
 ('of our', 28),
 ('and the', 26),
 ('puerto rico', 26),
 ('we will', 25),
 ('on the', 24),
 ('for our', 23)]

### Look at the ngrams for Bernie Sanders

In [14]:
# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('https co', 429),
 ('health care', 139),
 ('of the', 66),
 ('bernie sanders', 53),
 ('in the', 49),
 ('to the', 44),
 ('for the', 43),
 ('we must', 43),
 ('we need', 39),
 ('we are', 37),
 ('is not', 36),
 ('for all', 35),
 ('millions of', 35),
 ('donald trump', 33),
 ('going to', 33),
 ('is the', 31),
 ('on the', 30),
 ('this country', 30),
 ('and the', 28),
 ('should be', 28)]

## Processing the tweets and building a model

---

To do classfication we will need to convert the tweets into a set of features.

**You will need to:**
- Vectorize input text data.
- Intialize a model (try Logistic regression).
- Train / Predict / cross-validate.
- Evaluate the performance of the model.

> **Bonus:** you may have noticed that there are website links in the tweets. What additional preprocessing steps can you do before building the model?


In [19]:
# BONUS
# Using the textacy package to do some more comprehensive preprocessing
# http://textacy.readthedocs.io/en/latest/
import numpy as np
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [16]:
print(tweet_text[0:3])

[ 'Report: “ANTI-TRUMP FBI AGENT LED CLINTON EMAIL PROBE”  Now it all starts to make sense!'
 'People who lost money when the Stock Market went down 350 points based on the False and Dishonest reporting of Bria… https://t.co/4MT8wja1pn'
 'After years of Comey, with the phony and dishonest Clinton investigation (and more), running the FBI, its reputatio… https://t.co/gdbdgXZ57e']


In [20]:
print(clean_text[0:3])

['report antitrump fbi agent led clinton email probe now it all starts to make sense', 'people who lost money when the stock market went down 350 points based on the false and dishonest reporting of bria url', 'after years of comey with the phony and dishonest clinton investigation and more running the fbi its reputatio url']


In [21]:
# target is the handle.
# make trump 1 and sanders 0
y = tweets['handle'].map(lambda x: 1 if x == 'Donald J. Trump' else 0).values

print(np.mean(y))

0.498746867168


In [22]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(ngram_range=(1,4), max_features=2000)
X = tfv.fit_transform(clean_text).todense()

print(X.shape)

(1995, 2000)


In [23]:
# cross-validate the accuracy:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=10)

print(accuracies)
print(np.mean(accuracies))

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X, y)


[ 0.85        0.905       0.88        0.89        0.89        0.88944724
  0.90954774  0.88944724  0.87939698  0.86432161]
0.884716080402


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
# Very good accuracy considering the baseline is 50%

## Check the predicted probability for a random Sanders and Trump tweet
---

Below are provided a couple of tweets from both Sanders and Trump. I'm sure you can figure out on your own which one is which.

Estimate the predicted probability of being trump for the two tweets.

In [24]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!"
]

############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

Xtest = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
estimator.predict_proba(Xtest)

array([[ 0.77807224,  0.22192776],
       [ 0.32989667,  0.67010333]])

In [43]:
# The 1st column is probability of being Bernie, and 2nd Trump. The classifier is getting it right.

## Independent practice questions

---

### 1. Pull tweets for some new users.

Experiment with using more data.  The API will not like it if you blow through their limits - be careful.  Try to grab only what you need one time, then work on the copy of the objects that are returned.  

> Read the documentation about rate limits and see if you can get enough without hitting the rate limit.  Are there any options available in the API to avoid such a problem?

**Pull tweets for more than two different users of your choice.**

In [25]:
# We deviate from trump / sanders using student tweets here to illustrate the NLP pipeine with twitter data

twitter_handles = ["dril", "LaziestCanine",'ch000ch']
tweets = {}

for twitter_handle in twitter_handles:
    print("Mining tweets for: {}".format(twitter_handle))
    miner = TweetMiner(twitter_keys, api, result_limit=500)
    tweets[twitter_handle] = miner.mine_user_tweets(user=twitter_handle, max_pages=10)


Mining tweets for: dril
Mining tweets for: LaziestCanine
Mining tweets for: ch000ch


In [26]:
multi = pd.DataFrame(tweets['dril'])
multi = multi.append(pd.DataFrame(tweets['LaziestCanine']))
multi = multi.append(pd.DataFrame(tweets['ch000ch']))

print(multi.shape)

(5992, 6)


In [27]:
multi.handle.value_counts()

wint        1998
Lazy dog    1998
chuuch      1996
Name: handle, dtype: int64

### 2. Build a multi-class classification model to distinguish between the users.

Try a new type of model than we used before.

In [28]:
tweet_text = multi['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

y = multi['handle'].map(lambda x: 0 if x == 'wint' else 1 if x == 'Lazy dog' else 2).values

In [29]:
tfv = TfidfVectorizer(ngram_range=(1,3), max_features=2500)
X = tfv.fit_transform(clean_text)

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

rf = RandomForestClassifier(n_estimators=250, verbose=1)
knn = KNeighborsClassifier(n_neighbors=7)

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    5.9s finished


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [32]:
# Random forest score:
print('RF: {}'.format(rf.score(X_test, y_test)))
print('KNN: {}'.format(knn.score(X_test, y_test)))

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.2s finished


RF: 0.7018909899888766
KNN: 0.40378197997775306


In [33]:
# Baseline score:
multi.handle.value_counts() / multi.shape[0]

wint        0.333445
Lazy dog    0.333445
chuuch      0.333111
Name: handle, dtype: float64

In [34]:
rf_yhat = knn.predict(X_test)

### 3. Make a confusion matrix and classification report.

In [35]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, rf_yhat))

             precision    recall  f1-score   support

          0       0.52      0.22      0.31       636
          1       0.38      0.73      0.50       584
          2       0.40      0.28      0.33       578

avg / total       0.44      0.40      0.38      1798



In [36]:
# Confusion Matrix
print(confusion_matrix(y_test, rf_yhat))

[[140 346 150]
 [ 66 425  93]
 [ 61 356 161]]


### 4. What is the most and least "distinctive" tweets for each user?

To find this, identify the tweet that has the highest (correct) predicted probability of being that user's tweet for each user.

In [37]:
rf.fit(X, y)

pp = rf.predict_proba(X)

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    9.7s finished
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.6s finished


In [38]:
pp[0:5]

array([[ 0.928,  0.02 ,  0.052],
       [ 0.848,  0.008,  0.144],
       [ 0.836,  0.092,  0.072],
       [ 0.724,  0.084,  0.192],
       [ 0.876,  0.012,  0.112]])

In [39]:
pp = pd.DataFrame(pp, columns=['dril_pp', 'laziestcanine_pp', 'ch000ch_pp'])

In [40]:
print(multi.shape)
print(pp.shape)

(5992, 6)
(5992, 3)


In [41]:
tweets_pp = pd.concat([multi.reset_index(), pp.reset_index()], axis=1)
tweets_pp.head(2)

,index,created_at,handle,mined_at,retweet_count,text,tweet_id,index,dril_pp,laziestcanine_pp,ch000ch_pp
0,0,Sun Dec 03 23:12:50 +0000 2017,wint,2017-12-04 01:01:34.631592,5399,getting pissed off at the idea of someone goin...,937459644229828608,0,0.928,0.020,0.052
1,1,Sun Dec 03 15:35:19 +0000 2017,wint,2017-12-04 01:01:34.631604,1225,#WorldwideHandsomeDay looking dumb as a dog in...,937344504872558593,1,0.848,0.008,0.144


In [42]:
print('Most dril: {}'.format(tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=False).text.values[0]))
print('Least dril: {}'.format(tweets_pp[tweets_pp.handle == 'wint'].sort_values('dril_pp', ascending=True).text.values[0]))

Most dril: @HarshilShah1910 shut the fuck up
Least dril: @antonwheel thanks


In [43]:
print('Most LaziestCanine: {}'.format(tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=False).text.values[0]))
print('Least LaziestCanine: {}'.format(tweets_pp[tweets_pp.handle == 'Lazy dog'].sort_values('laziestcanine_pp', ascending=True).text.values[0]))

Most LaziestCanine: my body is my journal and my tattoos are my story https://t.co/k4F1rUFzn5
Least LaziestCanine: @Jaclyn_Mariee_ lmao tru


In [44]:
print('Most chuuch: {}'.format(tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=False).text.values[0]))
print('Least chuuch: {}'.format(tweets_pp[tweets_pp.handle == 'chuuch'].sort_values('ch000ch_pp', ascending=True).text.values[0]))

Most chuuch: @me_db lol
Least chuuch: https://t.co/WwvtlPWQMC
